# Imports and Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
import pathlib as pl
import spacy
from icecream import ic

plt.style.use('ggplot')
ic.configureOutput(includeContext=True)

# Getting Sampledata

In [ ]:
file_path = pl.Path(r"C:\Users\milit\Documents\python\Data_Analytics\ProjectWoche\Data\trip_advisor_reviews.zip")
data = pd.read_csv(file_path, compression="zip")
data.head()